## 构造 FaissDB

### 从 VectorDB 添加文本

In [2]:
from illufly.rag import FaissDB, TextEmbeddings

# 文档嵌入
db = FaissDB(embeddings=TextEmbeddings())
db.add("普鸿是一家做数字消防业务的公司")
db.add("幻蝶是一家AI技术公司")

# 查询
db("普鸿是啥？", top_k=5, verbose=True)

[INFO] 查询到2条结果


[Document(text="普鸿是一家做数字消防业务的公司", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary']),
 Document(text="幻蝶是一家AI技术公司", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary'])]

In [3]:
db.knowledge.get_meta_list()

{'total': 2,
 'total_pages': 1,
 'current_page': 1,
 'items': [{'id': '20241208-27932-0002-3656',
   'summary': '普鸿是一家做数字消防业务的公司',
   'tags': [],
   'source': ''},
  {'id': '20241208-27932-0003-7400',
   'summary': '幻蝶是一家AI技术公司',
   'tags': [],
   'source': ''}],
 'filters': {'tags': [], 'match_all_tags': True}}

### 从 VectorDB 添加本地文件

In [8]:
from illufly.rag import FaissDB, TextEmbeddings

# 文档嵌入
db = FaissDB(embeddings=TextEmbeddings())

# 从目录加载
db.knowledge.import_files("__DOCS__")
db.rebuild_index()
# 查询
db("披萨要烤几分钟？", top_k=2)

[Document(text="**Question:**<br>如何在家里制作美味的比萨？<br><br>**Answer:**<br>首先准备面团，然后加入番茄酱，放上你喜欢的配料，最后撒上奶酪，放入预热至200度的烤箱中烤约15分钟即可。", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary']),
 Document(text="**Question:**<br>自己动手做比萨需要注意哪些步骤？<br><br>**Answer:**<br>注意面团发酵时间，酱料的选择，配料的新鲜程度以及烘烤温度和时间。", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary'])]

### 使用本地知识库

In [2]:
from illufly.rag import FaissDB, TextEmbeddings
from illufly.io import LocalFileKnowledge

# 文档嵌入
db = FaissDB(knowledge=LocalFileKnowledge("__KG__"), embeddings=TextEmbeddings())

# 从目录加载
db.knowledge.import_files("__DOCS__")
db.rebuild_index()
# 查询
db("披萨要烤几分钟？", top_k=2)

[Document(text="**Question:**<br>如何在家里制作美味的比萨？<br><br>**Answer:**<br>首先准备面团，然后加入番茄酱，放上你喜欢的配料，最后撒上奶酪，放入预热至200度的烤箱中烤约15分钟即可。", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary']),
 Document(text="**Question:**<br>自己动手做比萨需要注意哪些步骤？<br><br>**Answer:**<br>注意面团发酵时间，酱料的选择，配料的新鲜程度以及烘烤温度和时间。", meta=['id', 'distance', 'score', 'source', 'raw_meta', 'tags', 'summary'])]

In [4]:
db.knowledge.directory

PosixPath('__KG__')

## 相似性比较实验

1. 如果文本完全相同，距离就是 (0)
2. 如果包含相同文本，混入没有实体的文本，即使很多，距离仍相近 (0.29)
3. 如果包含相同文本，混入其他实体后的文本，哪怕很少，距离依然增加 (0.43)
4. 如果表达接近，且实体不多，依然距离较近 (0.34)
5. 如果表达接近，但混入文本多，实体也多，距离就非常大 (0.94)

In [11]:
from illufly.rag import FaissDB, TextEmbeddings

# 文档嵌入
db = FaissDB(embeddings=TextEmbeddings())
db.add("普鸿是一家做数字消防业务的公司")
db.add("普鸿是一家做数字消防业务的公司，在广州做消防业务")
db.add("普鸿是一家做数字消防业务的公司，在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点")
db.add("普鸿是一家做数字消防业务的公司" + "在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点"*10)
db.add("普鸿是一家做数字消防业务的公司, 幻蝶是一家AI技术公司")
db.add("普鸿在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点")
db.add("普鸿做消防业务, 幻蝶做AI的，鸿蒙做政企服务")
db.add("2024年新成立的，主要股东有广州鸿蒙信息科技有限公司，普鸿是一家做数字消防业务的公司，是2024年新成立的，主要股东有广州鸿蒙信息科技有限公司")
db.add("幻蝶的AI很厉害"*15 + "消防业务")
db.add("幻蝶的AI很厉害"*15)

# 查询
for x in db("普鸿是一家做数字消防业务的公司", top_k=10, verbose=True):
    print(x.meta['distance'], ">>", x.text)

[INFO] 查询到10条结果
0.0 >> 普鸿是一家做数字消防业务的公司
0.07940569519996643 >> 普鸿是一家做数字消防业务的公司，在广州做消防业务
0.17805656790733337 >> 普鸿是一家做数字消防业务的公司，在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.29316282272338867 >> 普鸿是一家做数字消防业务的公司在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.3356439471244812 >> 普鸿在广州做消防业务，大概是2024年成立，公司业绩一直很好，全国各地开展新业务试点
0.3436765670776367 >> 普鸿是一家做数字消防业务的公司, 幻蝶是一家AI技术公司
0.3740360140800476 >> 2024年新成立的，主要股东有广州鸿蒙信息科技有限公司，普鸿是一家做数字消防业务的公司，是2024年新成立的，主要股东有广州鸿蒙信息科技有限公司
0.43234312534332275 >> 普鸿做消防业务, 幻蝶做AI的，鸿蒙做政企服务
0.9443018436431885 >> 幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶的AI很厉害幻蝶